# DEPENDENCIES AND FUNCTIONS

In [1]:
import numpy as np
from qiskit import QuantumCircuit, transpile
from qiskit.visualization import plot_histogram, circuit_drawer, plot_state_qsphere
from qiskit_aer import AerSimulator
from qiskit_aer.noise import pauli_error, NoiseModel, depolarizing_error
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.quantum_info import Statevector, Kraus, SuperOp
from qiskit_ibm_runtime import SamplerV2
from qiskit.result import sampled_expectation_value
from qiskit.quantum_info import Pauli

In [2]:
import networkx as nx

In [3]:
from qiskit.circuit import ParameterVector

In [4]:
import matplotlib.pyplot as plt

In [5]:
from qiskit.quantum_info import SparsePauliOp

In [6]:
from qiskit_aer.noise import (
    NoiseModel,
    QuantumError,
    ReadoutError,
    depolarizing_error,
    pauli_error,
    thermal_relaxation_error,
)

## BUILD THE NOISE MODEL 

In [7]:
from qiskit_ibm_runtime import EstimatorV2
# Create an empty noise model
noise_model = NoiseModel()
 
# Add depolarizing error to Hadamard (h) and CNOT (cx) gates
error_1qubit = depolarizing_error(0.0001, 1)  # 0.01% error on single-qubit gates
error_2qubit = depolarizing_error(0.01, 2)  # 1% error on two-qubit gates

# Apply the errors to Hadamard and CNOT gates
noise_model.add_all_qubit_quantum_error(error_1qubit, ['x', 'sx'])  
noise_model.add_all_qubit_quantum_error(error_2qubit, ['cx', 'ecr', 'cz'])  
 
# Print noise model info
print(noise_model)
estimator = EstimatorV2(backend=AerSimulator(noise_model=noise_model))

NoiseModel:
  Basis gates: ['cx', 'cz', 'ecr', 'id', 'rz', 'sx', 'x']
  Instructions with noise: ['sx', 'ecr', 'cz', 'x', 'cx']
  All-qubits errors: ['x', 'sx', 'cx', 'ecr', 'cz']


In [8]:
pm = generate_preset_pass_manager(optimization_level=3, basis_gates = ['x', 'sx', 'cx', 'rz'])

In [9]:
from scipy.optimize import minimize

In [10]:
import csv

In [11]:
import pandas as pd

In [12]:
def create_qaoa_circuit(p, n, G, pm):
    
    gamma = ParameterVector('γ',p)
    beta = ParameterVector('β',p)
    
    qc = QuantumCircuit(n, name='q')
    qc.h(range(n)) #superposition
    
    for i in range(p):
        for edge in G.edges(): #problem hamiltonian
            qc.rzz(gamma[i], edge[0], edge[1])
        for qubit in range(n): #mixer Hamiltonian
            qc.rx(2 * beta[i], qubit)
        if i != p-1:
            qc.barrier()
    return pm.run(qc)

In [13]:
def create_observables(graph, nodes):
    observables = []
    for edge in graph.edges():
        str = 'I'*nodes
        s_list = list(str)
        s_list[edge[0]]='Z'
        s_list[edge[1]]='Z'
        new_string = ''.join(s_list)
        observables.append(SparsePauliOp(new_string))
    return observables

In [14]:
def obtain_expval(params: list, qaoa: QuantumCircuit, observables: list, estimator) -> float:
    # execute the circuit
    job = estimator.run([(qaoa, observables, params)])
    result = job.result()[0]

    # sum up values
    value = sum(result.data.evs)

    return value

# COMPARING NAIVE AND TWO FOLD APPROACH

In [15]:
import joblib
gpr_models = joblib.load('gpr_models.pkl')
scaler = joblib.load('scaler.pkl')

### NAIVE APPROACH

In [16]:
import time

In [398]:
nodes = 8
t_depth = 5
adj_matrix = np.array([[0, 1, 1, 1, 1, 0, 0, 0],
[1, 0, 1, 1, 0, 1, 0, 1],
[1, 1, 0, 1, 1, 0, 0, 1],
[1, 1, 1, 0, 1, 1, 1, 0],
[1, 0, 1, 1, 0, 0, 1, 1],
[0, 1, 0, 1, 0, 0, 1, 0],
[0, 0, 0, 1, 1, 1, 0, 0],
[0, 1, 1, 0, 1, 0, 0, 0]
]
)
graph = nx.from_numpy_array(adj_matrix)
qaoa = create_qaoa_circuit(t_depth, nodes, graph, pm)
qaoa.measure_all()
observables = create_observables(graph, nodes)
init_params = [0]*qaoa.num_parameters
qaoa_params = qaoa.assign_parameters(init_params)
start = time.time()
ideal_res = minimize(
        obtain_expval, init_params, args=(qaoa.copy(), observables, estimator), method="L-BFGS-B"
)
end = time.time()
elapsed_time = end - start
print(f"Elapsed time: {elapsed_time} seconds")

Elapsed time: 16.00075602531433 seconds


### TWO_FOLD APPROACH

In [399]:
depth=1
qaoa = create_qaoa_circuit(depth, nodes, graph, pm)
qaoa.measure_all()
observables = create_observables(graph, nodes)
init_params = [0]*qaoa.num_parameters
qaoa_params = qaoa.assign_parameters(init_params)
ideal_res_new = minimize(
        obtain_expval, init_params, args=(qaoa.copy(), observables, estimator), method="L-BFGS-B"
)

In [400]:
arr = []
arr = arr + list(ideal_res_new.x)
arr.append(t_depth)
arr=np.array(arr)
arr=arr.reshape(1,-1)

In [401]:
#new_data = scaler.transform(arr)
data = arr[0]
predictions = []
for i in range(int(data[2]-1)):
    predictions.append(gpr_models[i].predict(arr))

for i in range(int(data[2]-1)):
    predictions.append(gpr_models[i+5].predict(arr))

In [402]:
init_params = predictions
init_params.insert(0, data[0])
init_params.insert(t_depth, data[1])
init_params = [item if not isinstance(item, np.ndarray) else item.item() for item in init_params]

In [403]:
qaoa = create_qaoa_circuit(t_depth, nodes, graph, pm)
qaoa.measure_all()
observables = create_observables(graph, nodes)
qaoa_params = qaoa.assign_parameters(init_params)
start = time.time()
ideal_res_new2 = minimize(
        obtain_expval, init_params, args=(qaoa.copy(), observables, estimator), method="L-BFGS-B"
)
end = time.time()
elapsed_time_new = end - start
print(f"Elapsed time: {elapsed_time_new} seconds")

Elapsed time: 12.163371562957764 seconds


In [404]:
print("number of iterations in naive approach : ", ideal_res.nit)
print("number of iterations in two-fold approach : ", ideal_res_new2.nit)
print("number of seconds in naive approach : ", elapsed_time)
print("number of seconds in two-fold approach : ", elapsed_time_new)

number of iterations in naive approach :  2
number of iterations in two-fold approach :  1
number of seconds in naive approach :  16.00075602531433
number of seconds in two-fold approach :  12.163371562957764


In [405]:
diff=100*(elapsed_time - elapsed_time_new)/elapsed_time
print(f'Reduction of time : {diff}%')

Reduction of time : 23.982519677729933%


In [406]:
exp_data = [ideal_res.nit, ideal_res_new2.nit, ideal_res.nit - ideal_res_new2.nit, elapsed_time, elapsed_time_new, elapsed_time - elapsed_time_new]

In [408]:
# Load the CSV file into a DataFrame
df = pd.read_csv('SampleResults_noise.csv')

# Modify the 13th row (index 12, since indexing starts from 0)
df.iloc[13] = exp_data  # Update with your desired values

# Save the modified DataFrame back to the CSV file
df.to_csv('SampleResults_noise.csv', index=False)


In [344]:
with open('SampleResults_noise.csv', mode='a', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(exp_data)

# INFERENCES

In [379]:
# Load the CSV file
df = pd.read_csv('SampleResults_noise.csv')
# Calculate the difference between the first two columns
df['Difference (nit)'] = df.iloc[:, 0] - df.iloc[:, 1]
# Insert the new column after the second column (index 2)
df.insert(2, 'Difference (nit)', df.pop('Difference (nit)'))
# Save the updated DataFrame back to the CSV file
df.to_csv('SampleResults_noise.csv', index=False)


In [380]:
# Load the CSV file
df = pd.read_csv('SampleResults_noise.csv')
# Calculate the difference between the first two columns
df['Difference (time)'] = df.iloc[:, 3] - df.iloc[:, 4]
# Save the updated DataFrame back to the CSV file
df.to_csv('SampleResults_noise.csv', index=False)


In [409]:
df = pd.read_csv('SampleResults_noise.csv')

tf_time = df['Two-fold (time in sec)']
tf_time_avg = tf_time.mean()

naive_time = df['Naive (time in sec)']
naive_time_avg = naive_time.mean()

naive_nit = df['Naive (nit)']
naive_nit_avg = naive_nit.mean()

tf_nit = df['Two-fold (nit)']
tf_nit_avg = tf_nit.mean()

red_nit = 100*(naive_nit_avg - tf_nit_avg)/(naive_nit_avg)
red_time = 100*(naive_time_avg - tf_time_avg)/(naive_time_avg)

print(f"The average reduction of time is: {red_time}%")
print(f"The average reduction of iterations is: {red_nit}%")

The average reduction of time is: 13.020004496759023%
The average reduction of iterations is: 8.163265306122456%


In [410]:
max_time = df['Difference (time)'].max()
max_it = df['Difference (nit)'].max()
max_time_row = df[df['Difference (time)'] == max_time]
max_it_row = df[df['Difference (nit)'] == max_it]

In [411]:
max_time, max_it

(10.54146957397461, 2)

In [412]:
max_time_row

,Naive (nit),Two-fold (nit),Difference (nit),Naive (time in sec),Two-fold (time in sec),Difference (time)
16,2,3,-1,28.938349,18.396879,10.54147


In [413]:
max_it_row

,Naive (nit),Two-fold (nit),Difference (nit),Naive (time in sec),Two-fold (time in sec),Difference (time)
7,2,0,2,21.123838,12.601596,8.522242
9,4,2,2,18.816135,15.477741,3.338393


In [415]:
mred_time = 100*(max_time/int(max_time_row['Naive (time in sec)']))
mred_nit = 100*(max_it/int(max_it_row['Naive (nit)'].iloc[1]))

print(f"The maximum reduction of time is: {mred_time}%")
print(f"The maximum reduction of iterations is: {mred_nit}%")

The maximum reduction of time is: 37.64810562133789%
The maximum reduction of iterations is: 50.0%


/tmp/ipykernel_52862/4101724127.py:1: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  mred_time = 100*(max_time/int(max_time_row['Naive (time in sec)']))
